Distancias entre textos. El último módulo permitirá, dados dos textos, calcular su distancia semántica. 

Para ello, evalúa diferentes alternativas y justifica la elección final tomada. 
La función se denominará texts_distance(text1: str, text2: str)